# Azure OpenAI Service Chat Completion 基礎
Chat Completion API は、GPT-35-Turbo および GPT-4 モデルと対話するための新しい専用 API です。この API は、これらのモデルにアクセスするための推奨される方法です。また、新しい GPT-4 モデルにアクセスする唯一の方法でもあります。

GPT-35-Turbo モデルと GPT-4 モデルは、会話インターフェイス用に最適化された言語モデルです。これらのモデルの動作は、以前の GPT-3 モデルとは異なります。以前のモデルは**テキストインとテキストアウト**でした。つまり、プロンプト文字列を受け入れ、そのプロンプトを補完する結果を返しました。それとは対照的に、GPT-35-Turbo および GPT-4 モデルは**カンバセーションインとメッセージアウト**です。モデルは、チャットのような特定の形式でフォーマットされた入力を想定し、チャット内のモデルで記述されたメッセージを表す入力候補を返します。この形式はマルチターン会話専用に設計されていますが、チャット以外のシナリオにも適しています。

https://learn.microsoft.com/azure/ai-services/openai/how-to/chatgpt?tabs=python-new&pivots=programming-language-chat-completions

## 事前準備

この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる[承認済み](https://aka.ms/oai/access) Azure サブスクリプション
- Azure OpenAI Service への GPT-3.5 Turbo / GPT-4 モデルのデプロイメント。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI GPT-3.5 Turbo / GPT-4 モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## パッケージのインストール


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 必要なライブラリと環境変数のインポート


## Azure OpenAI の設定
接続情報はセキュリティ面から直接記述するよりも、環境変数や [dotenv](https://pypi.org/project/python-dotenv/) からロードする方法をおすすめします。

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

#これは、モデルをデプロイしたときにデプロイメントに選んだカスタム名に対応します。
#AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"

## Chat Completion 呼び出し
GPT-35-Turbo および GPT-4 モデルは、会話形式の入力を処理するように最適化されています。 `messages` 変数は、システム（`system`）、ユーザー（`user`）、アシスタント（`assistant`）によって示された会話内のさまざまな役割を持つ辞書型の配列を渡します。

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "system", "content": "日本の歴史について回答するアシスタントです。"},
    {"role": "user", "content": "鎌倉幕府第三代征夷大将軍の名前を教えて"}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
)

print(response.choices[0].message.model_dump_json(indent=2))

### すべてのレスポンス内容を出力


In [ ]:
print(response.model_dump_json(indent=2))

## 対話の継続
Chat Completion API は**ステートレスな API** であり対話の履歴を API 側では持ちません。そのため、対話の履歴を考慮した推論をさせる場合は以下のように過去の結果を配列に含めてリクエストを送信します。

- `"role": "assistant"`: API からの返却値
- `"role": "user"`: ユーザーからの入力値

In [ ]:
messages= [
    {"role": "system", "content": "日本の歴史について回答するアシスタントです。"},
    {"role": "user", "content": "鎌倉幕府第三代征夷大将軍の名前を教えて"},
    {"role": "assistant", "content": "鎌倉幕府第三代征夷大将軍の名前は源実朝（みなもとのさねとも）です。"},
    {"role": "user", "content": "源実朝の趣味といえば？"},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.model_dump_json(indent=2))

## システムメッセージ
システムメッセージ（`system`）は**配列の先頭**に含まれ、モデルに最初の指示を与えます。システムメッセージには、次のようなさまざまな情報を指定できます。

- アシスタントの簡単な説明
- アシスタントの性格的な特性
- アシスタントに従ってもらいたい手順またはルール
- FAQ からの関連する質問など、モデルに必要なデータまたは情報

ユースケースに合わせてシステムメッセージをカスタマイズすることも、単に基本的な指示を含めることもできます。システムメッセージまたはメッセージは省略可能ですが、最適な結果を得るために、少なくとも基本的なものを含めることをお勧めします。

### システム メッセージのフレームワークとテンプレートに関する推奨事項
https://learn.microsoft.com/azure/ai-services/openai/concepts/system-message

In [ ]:
messages= [
    {"role": "system", "content": "日本の歴史について簡潔に回答するアシスタントです。回答の最後には回答に関連のある絵文字を含めてください。"},
    {"role": "user", "content": "源義経が活躍した戦いといえば"}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## メッセージ
システムメッセージの後に、ユーザーとアシスタント間の対話を一連のメッセージとして含めることができます。

In [ ]:
messages= [
    {"role": "system", "content": "日本の歴史について簡潔に回答するアシスタントです。回答の最後には回答に関連のある絵文字を含めてください。"},
    {"role": "user", "content": "源義経が活躍した戦いといえば"},
    {"role": "assistant", "content": "源義経が活躍した戦いといえば、平治の乱（へいじのらん）が挙げられます。🏹🏯"},
    {"role": "user", "content": "違いますよ、源義経が活躍した戦いといえば屋島の戦いです"},
]

In [ ]:
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## 知識を与える
関連するデータや情報をシステムメッセージに含めて、モデルに会話の追加コンテキストを与えることもできます。含める必要がある情報の量が少ない場合は、システムメッセージの中にハードコーディングできます。モデルで認識する必要があるデータの量が多い場合は、Embeddings API、または Azure AI Search などの製品を使用して、クエリ時に最も関連性の高い情報を取得できます。

In [ ]:
system_message = """
日本の鎌倉時代の歴史について簡潔に回答するアシスタントです。

#ルール
- 出典から答えが推測できない場合は「わかりません」と答えてください。
- 出典に無い答えを勝手に推測しないでください。

Context:###
源義経: 源義経-Wikipedia.pdf
一方、範頼の遠征軍は兵糧と兵船の調達に苦しみ、進軍が停滞してしまう。この状況を知った義経は後白河院に西国出陣を申し出てその許可を得た[注釈 12]。 
元暦2年（1185年）2月、新たな軍を編成した義経は、暴風雨の中を少数の船で出撃。通常3日かかる距離を数時間で到着し、讃岐国の瀬戸内海沿いにある平氏の
拠点屋島を奇襲し、山や民家を焼き払い、大軍に見せかける作戦で平氏を敗走させた（屋島の戦い）。
範頼も九州へ渡ることに成功し、最後の拠点である長門国彦島に拠る平氏の背後を遮断した。義経は水軍を編成して彦島に向かい、3月24日（西暦4月）の壇ノ浦の戦い
で勝利して、ついに平氏を滅ぼした。宿願を果たした義経は法皇から戦勝を讃える勅使を受け、一ノ谷、屋島以上の大功を成した立役者として、平氏から
取り戻した鏡璽を奉じて4月24日京都に凱旋する。
###

質問:
"""

messages= [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "源義経が活躍した戦いといえば"},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## Chat Completion を使用した Few-shot Learning
モデルに Few-shot の例を与えることもできます。Few-shot Learning の手法は、プロンプト形式が新しくなったため若干変更されています。ユーザーとアシスタントの間の一連のメッセージを、Few-shot の例としてプロンプトに含められるようになりました。これらの例を使用すると、一般的な質問に対する回答をシード処理してモデルを事前処理したり、特定の動作をモデルに教えたりすることができます。



In [ ]:
messages= [
    {"role": "system", "content": "日本の歴史について歴史家風に回答するアシスタントです。回答の最後には回答に関連のある絵文字を含めてください。"},
    {"role": "user", "content": "源義経が活躍した戦いといえば"},
    {"role": "assistant", "content": "源義経が活躍した戦いといえば屋島の戦いじゃよ🏹🚣‍♀️"},
    {"role": "user", "content": "源実朝の趣味といえば？"},
    {"role": "assistant", "content": "和歌を詠むことじゃよ📜🖋"},
    {"role": "user", "content": "鎌倉時代の武士の暮らしといえば？"},
    {"role": "assistant", "content": "武家社会の成立とともに、武士は武家としての生活を送るようになったんじゃ🏯🏹⚔️"},
    {"role": "user", "content": "源頼家の趣味といえば？"},
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages= messages,
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

print(response.choices[0].message.content)

## モデル別トークン数計測方法
OpenAI の [tiktoken](https://github.com/openai/tiktoken) ライブラリを使用して トークン数を計測します。
tiktoken ライブラリをインストールするには以下のコマンドを実行します。

`!pip install tiktoken --upgrade`

In [ ]:
import tiktoken

# "gpt-3.5-turbo-0613",
# "gpt-3.5-turbo-16k-0613",
# "gpt-4-0314",
# "gpt-4-32k-0314",
# "gpt-4-0613",
# "gpt-4-32k-0613",
model = "gpt-3.5-turbo-0613"
value = "源義経が活躍した戦いといえば"

try:
    encoding = tiktoken.encoding_for_model(model)
except KeyError:
    print("Warning: model not found. Using cl100k_base encoding.")
    encoding = tiktoken.get_encoding("cl100k_base")

print("Tokens:",len(value))
print(encoding.encode(value))
